## Семинар 10

Программа минимум на сегодня: области видимости, три слова про ООП, классы, наследование, перегрузка операторов

## Области видимости

In [89]:
# что не так с этим кодом?
a = 10

def foo():
    global a
    print(a)

foo()

10


In [92]:
# что не так с этим кодом?
a = 10

def foo():
    global a
    a += 1
    print(a)

foo()
print(a)

11
11


In [93]:
# пример отсюда: https://stackoverflow.com/questions/1261875/python-nonlocal-statement

x = 0
def outer():
    x = 1
    def inner():
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 1
global: 0


In [100]:
# пример отсюда: https://stackoverflow.com/questions/1261875/python-nonlocal-statement

def call_once(f):
    print("Code 1")
    called = False
    def wrap(*args, **argv):
        print("Code 2")
        nonlocal called
        if not called:
            called = True
            f(*args, **argv)
    return wrap
        

@call_once
def foo():
    print("Hi!")

Code 1


In [101]:
foo()
foo()

Code 2
Hi!
Code 2


In [102]:
# пример отсюда: https://stackoverflow.com/questions/1261875/python-nonlocal-statement

x = 0
def outer():
    x = 1
    def inner():
        nonlocal x
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 2
global: 0


In [86]:
# пример отсюда: https://stackoverflow.com/questions/1261875/python-nonlocal-statement

x = 0
def outer():
    x = 1
    def inner():
        global x
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

inner: 2
outer: 1
global: 2


## ООП

![Three rules](three-rules.png)

## class

In [123]:
class CounterInterface():
    def request(self):
        raise NotImplementedError("request is not implemented")
        
    def get_counter(self):
        raise NotImplementedError("get_counter is not implemented")

In [126]:
class CounterClass(CounterInterface):
    def __init__(self):
        self.requests_count = 0
    
    def request(self):
        self.requests_count += 1
    
    def get_counter(self):
        return self.requests_count

In [127]:
import unittest

class TestCounterClass(unittest.TestCase):
    def test_simple(self):
        counter = CounterClass()
        self.assertEqual(counter.get_counter(), 0)
        counter.request()
        self.assertEqual(counter.get_counter(), 1)
        
unittest.main(argv=["", "TestCounterClass"], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Сейчас код юниттестов должен был стать намного понятнее.

In [132]:
class CounterClass2(CounterClass):
    def request(self):
        self.requests_count += 2

In [133]:
import unittest

class TestCounterClass2(unittest.TestCase):
    def test_simple(self):
        counter = CounterClass2()
        self.assertEqual(counter.get_counter(), 0)
        counter.request()
        self.assertEqual(counter.get_counter(), 2)
        
unittest.main(argv=["", "TestCounterClass2"], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Вопрос, а что делать, если я хочу переопрелить конструктор.

In [140]:
class A():
    count = 1
    def __init__(self, n):
        print(self.count)
        self.count = n
        self.bubu = 2
    def request(self):
        self.count += 1
        self.bubu = 2


In [145]:
c = A(3)
print(c.count)
print(c.bubu)

1
3
2


In [150]:
class B():
    l = list()
    def __init__(self, n):
        self.l.append(n)


In [152]:
b = B(1)
b = B(2)
B.l
b.l

[1, 2, 1, 2]

In [154]:
c = B(3)
c.l

[1, 2, 1, 2, 3, 3]

In [162]:
class CounterClass3(CounterClass2):
    def __init__(self, n):
        super(CounterClass3, self).__init__()
        self.requests_count = n

In [163]:
import unittest

class TestCounterClass3(unittest.TestCase):
    def test_simple(self):
        counter = CounterClass3(3)
        self.assertEqual(counter.get_counter(), 3)
        counter.request()
        self.assertEqual(counter.get_counter(), 5)
        
unittest.main(argv=["", "TestCounterClass3"], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Полезная тема `isinstance`. Позволяет писать функции и методы, которые работают с разными типами по-разному.

In [53]:
a = CounterClass2()

In [54]:
print(f"isinstance(a, CounterClass3) = {isinstance(a, CounterClass3)}")
print(f"isinstance(a, CounterClass2) = {isinstance(a, CounterClass2)}")
print(f"isinstance(a, CounterClass) = {isinstance(a, CounterClass)}")
print(f"isinstance(a, CounterInterface) = {isinstance(a, CounterInterface)}")

isinstance(a, CounterClass3) = False
isinstance(a, CounterClass2) = True
isinstance(a, CounterClass) = True
isinstance(a, CounterInterface) = True


Красткое введение в исключения

In [4]:
raise Exception("some error")

Exception: some error

In [18]:
def raising_func():
    raise Exception("some error")
    
def value_error_func():
    raise ValueError("some value error")
    
def usual_func():
    return 0

In [19]:
import traceback

try:
    raising_func()
except Exception as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-19-7c506dfbe47f>", line 4, in <module>
    raising_func()
  File "<ipython-input-18-9219a0160b6a>", line 2, in raising_func
    raise Exception("some error")
Exception: some error


In [20]:

try:
    usual_func()
except Exception as e:
    traceback.print_exc()
else:
    print("OK")

OK


Вызывается первый подходящий обработчик

In [22]:
try:
    value_error_func()
except ValueError as e:
    print("value error")
    traceback.print_exc()
except Exception as e:
    print("any error")
    traceback.print_exc()
else:
    print("OK")

value error


Traceback (most recent call last):
  File "<ipython-input-22-f8dfe9c70f52>", line 2, in <module>
    value_error_func()
  File "<ipython-input-18-9219a0160b6a>", line 5, in value_error_func
    raise ValueError("some value error")
ValueError: some value error


Переопределение операторов.

Вам в большинстве случаев понадобятся только:
`__repr__`, `__str__`, [базовые операторы](https://docs.python.org/3/reference/datamodel.html#basic-customization), [операторы числовых типов](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types), [доступ к атрибутам](https://docs.python.org/3/reference/datamodel.html#customizing-attribute-access) и может быть [контексты](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers)

In [2]:
class AlwaysBigger():
    def __ge__(self, other):
        return True
    
    def __gt__(self, other):
        return True

In [3]:
very_big = AlwaysBigger()

In [70]:
print(f"very_big > 10 = {very_big > 10}")
print(f"very_big > 10**100 = {very_big >= 10**100}")
print(f"very_big > 'lol' = {'lol' < very_big}")

very_big > 10 = True
very_big > 10**100 = True
very_big > 'lol' = True


Вопрос, как же быть, что мы пытаемся сравнивать и слева и справа и у нас получается?

Напишите класс, который умеет красиво выводиться функцией `print`, для этого вам нужно переопределить функцию `__repr__`.

In [71]:
class Printable():
    def __init__(self, first, second):
        self.first = first
        self.second = second
        
    def __repr__(self):
        # your code here
        return ""
    
    def __str__(self):
        # your code here
        return ""

In [72]:
print(Printable(1, 2))

Напишите свой реализацию `Counter`, который считает количество различных объектов, которые в него добавили, сделайте так, чтобы с вашим классом работал оператор `in`. [сслыка](https://docs.python.org/3/reference/datamodel.html#object.__contains__)

In [73]:
# код здесь

In [74]:
# тесты здесь

Напишите свою реализацию defaultdict. Вам понадобится вот [этот метод](https://docs.python.org/3/reference/datamodel.html#object.__missing__)

In [75]:
# код здесь

In [76]:
# тесты здесь

Напишите класс, который считает сколько раз его вызывали. 
Вам понадобится метод [`__call__`](https://docs.python.org/3/reference/datamodel.html#object.__call__)

In [87]:
# код здесь

In [88]:
# тесты здесь

Напишите контекст, который на создание выводит "я начал", а на финализацию "я закончил".

In [1]:
# код здесь

Напишите класс декоратор. То есть callable класс, который принимает в качестве аргумента функцию. Подойдет любой декоратор из упражнений про декораторы.

In [ ]:
# код здесь

Напишите класс двумерного вектора, который умеет складываться, вычитаться и скалярно умножаться на число и другой вектор. А еще должен уметь красиво выводиться.

In [ ]:
# код здесь

In [ ]:
# тесты здесь

Если вы дошли до сюда, то давайте начинать писать игру Сапер. У вас будут следующие шаги.

1. Научиться генерировать поле размера N с K бомбами. N и K передаются аргументами командной строки. 
2. Научиться считывать с клавиатуры один символ и оторажать его на поле. Движение будет стрелочками, а открытие клетки enter.
3. Научиться отображать результат нажатия.

Задача очень неформально поставлена. Если есть проблемы как подступиться, спрашивайте.

Начните с создание Python проекта в PyCharm.